In [3]:
from pysr import PySRRegressor

In [1]:
import numpy as np
import pandas as pd

X = np.array([[1], [2], [3], [4], [5]])
y = np.array([3.1416, 12.5664, 28.2743, 50.2655, 78.5398])

pd.DataFrame(columns=["x0", "y"], data=np.column_stack([X, y]))

,x0,y
0,1.0,3.1416
1,2.0,12.5664
2,3.0,28.2743
3,4.0,50.2655
4,5.0,78.5398


In [2]:
from pysr import PySRRegressor

model = PySRRegressor(
  niterations=40,  # < Increase me for better results
  binary_operators=["+", "*", "-", "/"],
  unary_operators=[
    "cos",
    "exp",
    "sin",
    "inv(x) = 1/x",
    # ^ Custom operator (julia syntax)
  ],
  extra_sympy_mappings={"inv": lambda x: 1 / x},
  # ^ Define operator for SymPy as well
  elementwise_loss="loss(prediction, target) = (prediction - target)^2",
  # ^ Custom loss function (julia syntax)
)

model.fit(X, y)

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


/Users/alialh/Development/AI Lab/DSO/.venv/lib/python3.8/site-packages/pysr/sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Compiling Julia backend...


[ Info: Started!


PySRRegressor.equations_ = [
	   pick      score                              equation          loss  \
	0         0.000000                                    x0  1.660725e+03   
	1         0.508235                               exp(x0)  9.990197e+02   
	2         2.156951                        x0 * 12.851971  1.155639e+02   
	3  >>>>  13.066608                  x0 * (3.141592 * x0)  5.167863e-10   
	4         0.067141  (x0 * (3.141592 * x0)) + 8.121609e-6  4.518483e-10   
	
	   complexity  
	0           1  
	1           2  
	2           3  
	3           5  
	4           7  
]

In [ ]:
print(model.equations["equation"][9])

## Feynman Dataset

In [10]:
def generate_column_names(num_columns):
  """
  Generates column names as x1, x2, ..., y based on the number of columns.
  
  Parameters:
      num_columns (int): Total number of columns in the DataFrame.
  
  Returns:
      list: List of column names.
  """
  if num_columns < 2:
      raise ValueError("There should be at least one input variable and one target variable.")
  
  input_columns = [f'x{i}' for i in range(1, num_columns)]
  input_columns.append('y')  # Assuming the last column is the target variable
  return input_columns

In [11]:
def load_single_file(file_path):
  """
  Loads a single dataset file into a pandas DataFrame with appropriate column names.
  
  Parameters:
      file_path (str): Path to the dataset file.
  
  Returns:
      pandas.DataFrame: Loaded DataFrame with column headers.
  """
  try:
    # Detect the delimiter
    delimiter = "\s+"
    
    # Read the file without headers
    df = pd.read_csv(file_path, sep=delimiter, header=None, engine='python')
    
    # Generate column names
    column_names = generate_column_names(df.shape[1])
    df.columns = column_names
    
    return df
  except Exception as e:
    print(f"Error loading {file_path}: {e}")
    return None

In [17]:
import os
import pandas as pd

# Answer key for the Feynman equations
equations = pd.read_csv("datasets/FeynmanEquations.csv")

# Path to your dataset directory
dataset_dir = 'datasets/Feynman_with_units/'
# dataset_files = os.listdir(dataset_dir)
dataset_files = ["I.6.2a"]

# Iterate through each file in the directory
# for file_name in dataset_files:
#   file_path = os.path.join(dataset_dir, file_name)
  
#   # Attempt to read as comma-separated; fallback to space-separated if needed
#   df = load_single_file(file_path)
  
#   print(f"Loaded {file_name} with shape {df.shape}")
#   print(df.head())  # Display first few rows

filename = dataset_files[0]
file_path = os.path.join(dataset_dir, filename)
problem = load_single_file(file_path)
answer = equations[equations["Filename"] == filename]["Formula"].values[0]


In [18]:
answer

'exp(-theta**2/2)/sqrt(2*pi)'

In [20]:
X = problem[["x1"]].to_numpy()
y = problem["y"].to_numpy()

In [21]:
model = PySRRegressor(
  niterations=40,  # < Increase me for better results
  binary_operators=["+", "*", "-", "/"],
  batching=True,
  unary_operators=[
      "cos",
      "exp",
      "sin",
      "inv(x) = 1/x",
      "sqrt"
      # ^ Custom operator (julia syntax)
  ],
  turbo=True,
  progress=True,
  extra_sympy_mappings={"inv": lambda x: 1 / x},
  # ^ Define operator for SymPy as well
  elementwise_loss="loss(prediction, target) = (prediction - target)^2",
  # ^ Custom loss function (julia syntax)
)

model.fit(X, y)

/Users/alialh/Development/AI Lab/DSO/.venv/lib/python3.8/site-packages/pysr/sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Compiling Julia backend...


    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed SIMDTypes ──────────────────────── v0.1.0
   Installed CpuId ──────────────────────────── v0.3.1
   Installed OffsetArrays ───────────────────── v1.14.1
   Installed LayoutPointers ─────────────────── v0.1.17
   Installed CPUSummary ─────────────────────── v0.2.6
   Installed BitTwiddlingConvenienceFunctions ─ v0.1.6
   Installed VectorizationBase ──────────────── v0.21.70
   Installed LoopVectorization ──────────────── v0.12.171
   Installed IfElse ─────────────────────────── v0.1.1
   Installed PolyesterWeave ─────────────────── v0.2.2
   Installed CloseOpenIntervals ─────────────── v0.1.13
   Installed ThreadingUtilities ─────────────── v0.5.2
   Installed ManualMemory ───────────────────── v0.1.8
   Installed HostCPUFeatures ────────────────── v0.1.17
   Installed Static ─────────────────────────── v1.1.1
   Installed SLEEFPirates ───────────────────── v0.6.43
   Installed Comm


Expressions evaluated per second: 5.800e+02
Head worker occupation: 59.4%. This is high, and will prevent efficient resource usage. Increase `ncycles_per_iteration` to reduce load on head worker.
Progress: 58 / 600 total iterations (9.667%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.636e-01  1.594e+01  y = 0.47713
2           4.913e-03  3.506e+00  y = sin(0.078873)
3           2.296e-03  7.608e-01  y = 0.12887 / x₀
5           4.055e-04  8.669e-01  y = inv(sqrt(exp(exp(x₀))))
6           9.487e-05  1.453e+00  y = sin(inv(exp(1.3803 * x₀)))
7           8.779e-05  7.764e-02  y = sin(inv(exp(x₀ / cos(0.78315))))
8           4.430e-05  6.839e-01  y = (sin(sin(x₀)) / exp(x₀)) / x₀
9           2.481e-06  2.882e+00  y = (sin(x₀) / (exp(x₀) - -0.70019)) / x₀
10          2.471e-06  4.247e-03  y = sin((sin(x₀) / (0.68632 + exp(x₀))) / x₀)
11          2.444e-06  1.105e-0

PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                         0.47712913   
	1         3.505567                                    sin(0.07887315)   
	2         0.997645                                    0.16825546 / x0   
	3         1.202310                               0.58362263 / exp(x0)   
	4         2.745324                     -0.14735486 + (0.4085565 / x0)   
	5         0.346102                sin((0.3953635 / x0) + -0.13876131)   
	6         0.012037           sin((0.3953635 / x0) + sin(-0.13876131))   
	7         0.123063       (inv(exp(x0)) / inv(0.7867192)) - 0.04676809   
	8         2.164500            (sin(x0) / (exp(x0) - -0.7001883)) / x0   
	9         1.059205     sin(inv(exp(x0 * sqrt(x0)) - inv(-0.7060522)))   
	10        0.227204  sin(inv(exp(sqrt(x0) * x0) - (-0.49958715 / 0....   
	11        0.010655  sin(inv(exp(sqrt(x0) * x0) - ((x0 - x0) - 1.39...   
	12        0.505711  inv(exp(sqrt(x0) * x0) + (sqrt(sin(0.41910994)...   
	13        1.549286  sin(inv(sqrt(exp(((inv(x0) + x0) + (inv(x0) / ...   
	14  >>>>  0.166672  sin(sin(sin(inv((inv(sqrt(0.618324)) + exp(sqr...   
	
	            loss  complexity  
	0   1.636185e-01           1  
	1   4.913424e-03           2  
	2   1.811809e-03           3  
	3   5.444475e-04           4  
	4   3.496849e-05           5  
	5   2.473813e-05           6  
	6   2.444214e-05           7  
	7   2.161194e-05           8  
	8   2.481208e-06           9  
	9   8.603127e-07          10  
	10  6.854610e-07          11  
	11  6.710082e-07          13  
	12  4.046696e-07          14  
	13  8.595161e-08          15  
	14  3.735343e-08          20  
]

In [24]:
model.equations["equation"].iloc[14]

/Users/alialh/Development/AI Lab/DSO/.venv/lib/python3.8/site-packages/pysr/sr.py:1172: FutureWarning: PySRRegressor.equations is now deprecated. Please use PySRRegressor.equations_ instead.
  warnings.warn(


'sin(sin(sin(inv((inv(sqrt(0.618324)) + exp(sqrt(x0) / inv(x0))) / x0))) / x0) - 0.0009244262'